# Mobile price classification with multilayer perceptron
## Tensorflow
Erik Matovič and Jakub Horvat

[Dataset](https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification?select=train.csv)

Predict a price range indicating how high the price is

### 0. Imports

In [83]:
from utils import split_train_val
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
from sklearn.model_selection import train_test_split
from utils import check_null_values, print_sum_null, rescale, split_data
import numpy as np

In [84]:
# api = wandb.Api()
# sweep = api.sweep("xmatovice/uncategorized/1stel95e") #2hfg3dq6") #1kmqujzs")
#
# best_config = sweep.best_run().config
# pprint(best_config)
#
# wandb.init(config=best_config, project='uncategorized', name='NSIETE_zad1_final')
# config = wandb.config

In [85]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.reduce_sum(tf.random.normal([1000, 1000])))
print(tf.config.list_physical_devices('GPU'))


2.11.0
Num GPUs Available:  0
tf.Tensor(-2436.9355, shape=(), dtype=float32)
[]


In [86]:
wandb.login()
run=wandb.init(project="nsiete1")

### 1. Data Preprocessing

Based on [exploratory data analysis](EDA.ipynb) test set does not have target variable price_range. We split our dataset into train-dev-test. We have train and test sets, but we split test set by half to dev-test sets. We will rougly have train-dev-test 67%-16.5%-16.5%. 

In [87]:
# read datasets
df_train = pd.read_csv('../data/train.csv', sep=',')
df_test = df_train[-200:]
df_train = df_train[:-200]
print(df_train.shape,df_test.shape)
df_train


(1800, 21) (200, 21)


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,527,0,0.7,1,4,0,53,0.9,128,1,...,67,1924,1302,13,7,13,0,0,0,0
1796,793,1,2.8,1,1,1,64,0.4,179,4,...,818,1006,2296,12,0,5,1,1,0,1
1797,873,1,0.5,1,6,0,8,0.6,152,7,...,1257,1411,575,15,8,7,1,0,1,0
1798,722,0,1.1,0,12,0,12,1.0,93,6,...,138,1371,2359,13,6,3,0,0,0,1


In [88]:
#df_test = pd.read_csv('../data/test.csv', sep=',', index_col=0)
#df_test

In [89]:
#df_train.values

In [90]:
x_train = df_train.values[:, :-1]       #vsetko okrem price range
x_train = x_train.astype('float32')     # input data as floats
#x_train
y_train = df_train.values[:, -1]  # price range
y_train = y_train.astype('int')
print(y_train)

[1 2 2 ... 0 1 2]


In [91]:
x_test = df_test.values[:, :-1]       #vsetko okrem price range
x_test = x_test.astype('float32')     # input data as floats
#x_test
y_test = df_test.values[:, -1]  # price range
y_test = y_test.astype('int')
print(y_test)

[0 1 3 3 1 2 1 0 0 1 0 1 0 0 1 3 3 1 0 2 0 3 1 2 2 1 2 3 3 0 0 1 2 2 2 1 2
 1 0 2 3 0 0 3 0 3 0 0 3 3 0 3 0 0 0 0 3 3 2 0 0 0 0 3 0 2 3 2 3 3 2 3 1 0
 2 0 0 1 1 1 2 2 0 0 3 0 1 2 0 3 1 0 0 3 0 1 0 3 3 0 1 2 2 2 3 0 1 3 3 1 3
 0 1 0 0 2 1 0 2 0 3 1 0 2 1 2 1 3 0 2 3 1 1 1 2 1 3 2 2 2 2 3 1 0 0 0 0 2
 1 2 1 0 1 3 0 3 0 3 3 1 2 0 2 1 1 0 2 1 0 2 2 1 0 3 1 3 0 2 1 1 1 1 2 2 1
 3 1 0 1 1 0 3 0 3 3 0 2 3 0 3]


In [92]:
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.2) #,random_state=52
print(x_train.shape,x_val.shape,y_train.shape,y_val.shape)
#print(y_train," sad")
class_labels = [0, 1, 2, 3]
y_cat_train = tf.keras.utils.to_categorical(y_train, num_classes=len(class_labels))
y_cat_val = tf.keras.utils.to_categorical(y_val, num_classes=len(class_labels))
y_cat_test = tf.keras.utils.to_categorical(y_test, num_classes=len(class_labels))
print(y_train)

(1440, 20) (360, 20) (1440,) (360,)
[1 0 1 ... 2 0 3]


In [93]:
# x_test = df_test.values[:, :]       #vsetko okrem price range
# x_test = x_test.astype('float32')
# x_test

In [94]:
model = keras.Sequential([
    # keras.layers.Input(shape=(20,),name='input'),
    keras.layers.Dense(256,input_shape=(20,),activation='relu',name='l1'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(256,activation='relu',name='l2'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(256,activation='relu',name='l3'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(256,activation='relu',name='l4'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(4,activation='sigmoid',name='l5') #activation='sigmoid'
])


In [95]:
lr = 0.03648030109469176
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy','mse']
              )


In [ ]:
tf.debugging.enable_traceback_filtering()
history = model.fit(x_train,
                    y_cat_train,
                    batch_size=128,
                    epochs=100,
                    validation_data=(x_val,y_cat_val),
                    callbacks=[
                      WandbMetricsLogger(log_freq=5),
                      WandbModelCheckpoint("models")
                    ],
                    verbose=0)

INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.1s


INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.0s


INFO:tensorflow:Assets written to: models\assets


...

INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.1s


INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.1s


INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets
wandb: Adding directory to artifact (.\models)... Done. 0.1s


In [ ]:
import matplotlib.pyplot as plt

#Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# # Plot training & validation categorical accuracy
# plt.plot(history.history['categorical_accuracy'])
# plt.plot(history.history['val_categorical_accuracy'])
# plt.title('Model categorical accuracy')
# plt.ylabel('Categorical accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

In [ ]:
score=model.evaluate(x_test,y_cat_test)
predict = model.predict(x_test)
predict

In [ ]:
pom = []
for row in predict:
    a = max(row)
    b=np.where(row==a)[0][0]
    pom.append(b)
print(pom)

In [ ]:
y_test.size

In [ ]:
arr = pom - y_test
a = len(arr)
b = len(y_test)
c = a-b
print(arr,a,b)

In [ ]:
wandb.finish()
